In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from copy import copy
import sys; sys.path.append('../../../tidy3d')

import tmm
import tidy3d as td
from tidy3d import web

In [9]:
freq0 = 2e14
fwidth = freq0 / 10.0
freqs = [freq0]
wavelength = td.C_0 / freq0

theta = 0
bck_eps = 1.4**2
ds0 = [0.5, 0.25, 0.5, 0.25]
eps0 = [2**2, 1.8**2, 1.5**2, 1.9**2]
dl= 0.01

In [11]:
def normalize(arr):
    return arr / np.linalg.norm(arr)

def compute_error(truth, guess):
    return np.linalg.norm(truth - guess) / np.linalg.norm(truth)

def ds_to_tops(ds):
    slab_tops = []
    z_bottom = -np.sum(ds) / 2.
    for d in ds:
        z_top = z_bottom + d
        slab_tops.append(z_top)
        z_bottom = z_top
    return slab_tops

def tops_to_ds(tops):
    z_bottom = -tops[-1]
    d_bottom = tops[0] - z_bottom
    ds = [d_bottom]
    for i in range(len(tops)-1):
        ds.append(tops[i+1] - tops[i])
    return ds

tops0 = ds_to_tops(ds0)

In [4]:
def compute_T_tmm(eps, tops):
    ds = tops_to_ds(tops)
    eps_list = [bck_eps] + list(eps) + [bck_eps]
    n_list = np.sqrt(eps_list)
    d_list = [np.inf] + list(ds) + [np.inf]
    return tmm.coh_tmm("p", n_list, d_list, theta, wavelength)["T"]

def val_and_grad_tmm():

    T0 = compute_T_tmm(eps=eps0, tops=tops0)

    num_slabs = len(eps0)
    grad_tmm = np.zeros((2, num_slabs), dtype=float)
    args = np.stack((eps0, tops0), axis=0)
    delta = 1e-6

    # epsilon gradients
    for arg_index in range(2):
        for slab_index in range(num_slabs):
            grad = 0.0
            for pm in (-1, +1):
                args_num = args.copy()
                args_num[arg_index][slab_index] += delta * pm
                T_tmm = compute_T_tmm(args_num[0], args_num[1])
                grad += pm * T_tmm / 2 / delta        
            grad_tmm[arg_index][slab_index] = grad

    grad_eps, grad_top = grad_tmm

    grad_eps = normalize(grad_eps)
    grad_top = normalize(grad_top)
    
    return T0, (grad_eps, grad_top)

In [5]:
def make_sim_fwd():

    # geometry setup
    bck_medium = td.Medium(permittivity=bck_eps)

    space_above = 2
    space_below = 2

    length_x = 0.5
    center_x = 0.0
    length_y = 0.0
    length_z = space_below + sum(ds0) + space_above
    sim_size = (length_x, length_y, length_z)

    # make structures
    slabs = []
    tops0 = ds_to_tops(ds0)
    slab_ds = tops_to_ds(tops0)
    z_start = -tops0[-1]
    for (d, eps) in zip(slab_ds, eps0):
        slab = td.Structure(
            geometry=td.Box(
                center=[0, 0, z_start + d / 2],
                size=[td.inf, td.inf, d]
            ),
            medium=td.Medium(permittivity=eps),
        )
        slabs.append(slab)
        z_start += d

    # source setup
    gaussian = td.GaussianPulse(freq0=freq0, fwidth=fwidth)
    src_z = -length_z / 2 + 3 * space_below / 4

    source = td.PlaneWave(
        center=(center_x, 0, src_z),
        size=(td.inf, td.inf, 0),
        source_time=gaussian,
        direction="+",
        angle_theta=theta,
        angle_phi=0,
        pol_angle=0,
    )

    # boundaries
    boundary_x = td.Boundary.bloch_from_source(
        source=source, domain_size=sim_size[0], axis=0, medium=bck_medium
    )
    boundary_spec = td.BoundarySpec(x=boundary_x, y=td.Boundary.periodic(), z=td.Boundary.pml(num_layers=40))

    # monitors
    mnt_z = length_z / 2 - wavelength
    monitor_1 = td.DiffractionMonitor(
        center=[0.0, 0.0, mnt_z],
        size=[td.inf, td.inf, 0],
        freqs=freqs,
        name="diffraction",
        normal_dir="+",
    )

    monitor_2 = td.FieldMonitor(
        center=[0.0, 0.0, mnt_z],
        size=[td.inf, td.inf, 0],
        freqs=freqs,
        name="field",
    )

    # monitors to record the fields and permittivity needed for the gradient computation
    # they need to span 
    monitor_g1 = td.FieldMonitor(
        center=[0.0, 0.0, 0.0],
        size=[td.inf, td.inf, np.sum(ds0)],
        freqs=freqs,
        name="field_grad",
    )
    monitor_g2 = td.PermittivityMonitor(
        center=[0.0, 0.0, 0.0],
        size=[td.inf, td.inf, np.sum(ds0)],
        freqs=freqs,
        name="eps_grad",
    )

    # make simulation
    sim = td.Simulation(
        size=sim_size,
        grid_spec=td.GridSpec.uniform(dl=dl),
        structures=slabs,
        sources=[source],
        monitors=[monitor_1, monitor_2, monitor_g1, monitor_g2],
        run_time=50 / fwidth,
        boundary_spec=boundary_spec,
        medium=bck_medium,
        shutoff=1e-8,
    )
    return sim

def compute_T_fdtd(sim_data):
    amp = complex(sim_data["diffraction"].amps.sel(polarization="p").values)
    return abs(amp)**2
        

def make_sim_adj(sim_data_fwd):
    
    sim_fwd = sim_data_fwd.simulation
    
    amps = complex(sim_data_fwd["diffraction"].amps.sel(polarization="p").values)
    src_amp = 1j * np.conj(amps)
    src_time = td.GaussianPulse(freq0=freq0, fwidth=fwidth, amplitude=abs(src_amp), phase=np.angle(src_amp))
    
    # source needs to be centered at the simulation center_x, not the monitor one
    source_adj = td.PlaneWave(
        center=sim_fwd.get_monitor_by_name('diffraction').center,
        size=(td.inf, td.inf, 0),
        source_time=src_time,
        direction="-",
        angle_theta=theta,
        angle_phi=0,
        pol_angle=0,
    )
    
    sim_adj = sim_fwd.copy(update=dict(sources=[source_adj]))    

    # adjoint boundaries (bloch vector flips sign) because of source direction = "-"
    boundary_x = td.Boundary.bloch_from_source(
        source=source_adj, domain_size=sim_adj.size[0], axis=0, medium=sim_adj.medium
    )

    boundary_spec_adj = sim_fwd.boundary_spec.copy(update=dict(x=boundary_x))
    
    return sim_adj.copy(update=dict(boundary_spec=boundary_spec_adj))

def val_and_grad_fdtd():
    
    sim_fwd = make_sim_fwd()
    sim_data_fwd = web.run(sim_fwd, task_name="multilayer_fwd")
    T = compute_T_fdtd(sim_data_fwd)
    
    sim_adj = make_sim_adj(sim_data_fwd)
    sim_data_adj = web.run(sim_adj, task_name="multilayer_adj")

    # compute gradient w.r.t. slab permittivity and the top of the slab boundaries
    field_fwd = sim_data_fwd["field_grad"]
    field_adj = sim_data_adj["field_grad"]

    Exf = field_fwd.Ex
    Eyf = field_fwd.Ey
    Ezf = field_fwd.Ez
    Dzf = Ezf * sim_data_fwd["eps_grad"].eps_zz
    Exa = field_adj.Ex
    Eya = field_adj.Ey
    Eza = field_adj.Ez
    Dza = Eza * sim_data_adj["eps_grad"].eps_zz

    eps_list = [bck_eps] + eps0 + [bck_eps]
    length_x = sim_adj.size[0]
    
    nx = 100
    dx = length_x / nx
    
    xs = np.linspace(-length_x / 2 + dx / 2, length_x / 2 - dx/2, nx)
    # zs = Exf.z

    grad_top_adj = []
    grad_eps_adj = []

    for islab, slab in enumerate(sim_fwd.structures):

        ((_, _, zmin), (_, _, zmax)) = slab.geometry.bounds

        # epsilon gradient
        lz = zmax - zmin
        nz = 100
        dz = lz / nz
        zs = np.linspace(zmin+dz/2, zmax-dz/2, nz)
        
        ex_fwd = Exf.interp(x=xs, z=zs)
        ey_fwd = Eyf.interp(x=xs, z=zs)
        ez_fwd = Ezf.interp(x=xs, z=zs)
        
        ex_adj = Exa.interp(x=xs, z=zs)
        ey_adj = Eya.interp(x=xs, z=zs)
        ez_adj = Eza.interp(x=xs, z=zs)
        
        integrand_eps = ex_fwd * ex_adj + ey_fwd * ey_adj + ez_fwd * ez_adj
        grad_eps = dz * dx * np.sum(integrand_eps)
        grad_eps_adj.append(grad_eps)

        # top boundary gradient
        d_eps = eps_list[islab + 1] - eps_list[islab + 2]
        d_eps_inv = (1 / eps_list[islab + 1] - 1 / eps_list[islab + 2])

        ex_fwd = Exf.interp(x=xs, z=zmax)
        ex_adj = Exa.interp(x=xs, z=zmax) 
        ey_fwd = Eyf.interp(x=xs, z=zmax) 
        ey_adj = Eya.interp(x=xs, z=zmax)
        dz_fwd = Dzf.interp(x=xs, z=zmax) 
        dz_adj = Dza.interp(x=xs, z=zmax) 

        integrand = d_eps * (ex_fwd * ex_adj + ey_fwd * ey_adj)
        integrand -= d_eps_inv * (dz_fwd * dz_adj)

        grad = np.sum(integrand)
        grad_top_adj.append(grad)

    g_eps_adj = np.real(grad_eps_adj)
    grad_eps_adj = normalize(g_eps_adj)

    g_top_adj = np.real(grad_top_adj)
    grad_top_adj = normalize(g_top_adj)

    return T, (grad_eps_adj, grad_top_adj)

In [6]:
T_tmm, (grad_eps_tmm, grad_top_tmm) = val_and_grad_tmm()
T_fdtd, (grad_eps_fdtd, grad_top_fdtd) = val_and_grad_fdtd()

error_T = compute_error(T_tmm, T_fdtd)
error_eps = compute_error(grad_eps_tmm, grad_eps_fdtd)
error_top = compute_error(grad_top_tmm, grad_top_fdtd)

↓ monitor_data.hdf5 ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 61.4% • 524.3/854.2 kB • 659.2 kB/s • 0:00:01

[15:05:43] INFO     loading SimulationData from simulation_data.hdf5                                  ]8;id=545103;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/source/notebooks/../../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=399748;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/source/notebooks/../../../tidy3d/tidy3d/web/webapi.py#407\407]8;;\

In [7]:
print('\ntransmission:\n')
print(T_tmm)
print(T_fdtd)
print(f'error = {(error_T*100):.4f}%')
print('\ngradient EPSILON:\n')
print(grad_eps_tmm)
print(grad_eps_fdtd)
print(f'error = {(error_eps*100):.4f}%')
print('\ngradient TOP:\n')
print(grad_top_tmm)
print(grad_top_fdtd)
print(f'error = {(error_top*100):.4f}%')


transmission:

0.8752655778361093
0.8755003600625411
error = 0.0268%

gradient EPSILON:

[-0.47604842  0.72026671  0.08335161 -0.49764072]
[-0.47378827  0.72176256  0.08697163 -0.49701048]
error = 0.4566%

gradient TOP:

[-0.4387459   0.37791092 -0.5143388   0.63256697]
[-0.44575007  0.38445325 -0.52224879  0.61705654]
error = 1.9875%
